# 패키지 선언

In [1]:
# 패키지 불러오기

import requests
import bs4
from bs4 import BeautifulSoup
import os
import random
import hashlib
import urllib.request
import urllib
from urllib.parse import urlparse, urlencode, parse_qs
import pandas as pd
import csv
import re

from datetime import datetime, timedelta
from urllib import parse
import time
import requests
import numpy as np
import json
from random import *
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent

from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

ua = UserAgent()
userAgent = ua.random
print(userAgent)

options = Options()
options.add_argument(f"user-agent={userAgent}")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--proxy-server=socks5://127.0.0.1:9050')

executable_path = '/home/book_crawling/geckodriver' # chrome driver의 경로
driver = webdriver.Firefox(executable_path=executable_path, options=options)

Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


# 인터파크 크롤링

In [2]:
# 인터파크 크롤링 함수

def interpark_crawl(isbn):
    
    book = {'isbn' : isbn,
            '제목' : '없음',
            '저자' : '없음',
            '역자' : '없음',
            '출판일' : '없음',
            '출판사' : '없음',
            '별점' : '없음',
            '리뷰' : '없음',
            '카테고리' : '없음',
            '책소개' : '없음',
            '저자소개' : '없음',
            '추천사' : '없음',
            '목차' : '없음',
            '책속으로' : '없음',
            '출판사리뷰' : '없음',
            '판매지수' : '없음'
           }
    
    # isbn 검색, 첫번째 아이템 링크 열기
    search_page = requests.get('http://bsearch.interpark.com/dsearch/book.jsp?sch=all&query=' + isbn)
    list_soup = BeautifulSoup(search_page.text, 'html.parser')
    try:
        search_list = list_soup.find(class_='list_wrap')
        top_item_image = search_list.find(class_='bimgWrap')
        top_item_link = top_item_image.a['href']
        driver.get(top_item_link)
    except:
        return book

    # 상품 창에서 펼쳐보기 모두 클릭
    txt_contents = driver.find_elements_by_class_name('detail_txtContent')
    btns = driver.find_elements_by_class_name('bt_moreTxt')
    for btn in btns:
        try:
            btn.click()
        except:
            pass

    page_src = driver.page_source
    soup = BeautifulSoup(page_src, 'html.parser')

    # 제목
    try:
        title = soup.select_one('.titleWrap').h2.text
    except:
        title = '없음'
    book['제목'] = title

    # 저자, 역자
    try:
        book_info = soup.select_one('.bInfo_txt')
        infos = book_info.find_all('li')

        author = '없음'
        translaor = '없음'
        pub_date = '없음'
        publisher = '없음'
        for info in infos:
            if '저 : ' in info.text:
                author = info.text.replace('저 : ', '')
            if '역 : ' in info.text:
                translator = info.text.replace('역 : ', '')
            if '20' in info.text:
                pub_date = info.text.replace('발행 : ', '')
            if '출판사' in info.text:
                publisher = info.text.replace('출판사 : ', '')
    except:
        pass
    book['저자'] = author
    book['역자'] = translaor
    book['출판일'] = pub_date
    book['출판사'] = publisher

    # 별점
    try:
        rating_el = soup.select_one('.bStarGradeBox')
        rating = rating_el.text.replace('리뷰쓰기', '').strip()
    except:
        rating = '없음'
    book['별점'] = rating
    
    # 리뷰
    try:
        review_el = soup.select_one('.detailTitle2').select_one('.total_count')
        review = review_el.text.replace('(총 ', '').replace('건)', '')
    except:
        review = '없음'
    book['리뷰'] = review
    
    # 판매지수
    try:
        sales_el = driver.find_element_by_class_name('saleIndexWrap')
        sales_index = sales_el.text.replace('?', '').strip()
    except:
        sales_index = '없음'
    book['판매지수'] = sales_index

    # 카테고리
    try:
        field_el = soup.select_one('.fiedCell')
        field = field_el.text.replace('Home\n', '').replace('\n', ' > ').replace('>  >', '').strip()
    except:
        field = '없음'
    book['카테고리'] = field

    # 책소개, 출판사리뷰, 추천사, 책속으로, 목차   
    book_intro = '없음'
    pub_review = '없음'
    table = '없음'
    recommend = '없음'
    book_inside = '없음'
    
    try:
        for content in txt_contents:
            section = driver.execute_script("""
                return arguments[0].previousElementSibling
            """, content).text
            if '책소개' in section:
                book['책소개'] = content.text.replace('\n\n접어보기', '').strip()
            if '출판사' in section:
                book['출판사리뷰'] = content.text.replace('\n펼쳐보기', '').strip()
            if '추천사' in section:
                book['추천사'] = content.text.replace('\n\n접어보기', '').strip()
            if '목차' in section:
                book['목차'] = content.text
            if '본문' in section:
                book['책속으로'] = content.text.replace('\n\n접어보기', '').strip()
    except:
        pass       
    
    # 저자소개
    try:
        author_intro = []
        writer_info = soup.select('.writerInfo')
        for info in writer_info:
            if '[저]' in info.select_one('.writerName').text:
                author_name = info.select_one('.writerName').text.replace('[저]', '').strip() + '\n'
                info_text = info.p.text.replace('\n', '').strip()
                author_intro.append(author_name + info_text)
    except:
        author_intro = '없음'
    book['저자소개'] = author_intro    
        
    return book

# csv 파일 불러오기 & 크롤링

In [3]:
book_specs = pd.read_csv('./book_spec_korean_isbns.csv', low_memory=False, index_col=0)
isbns = book_specs['0'].tolist()
len(isbns)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


2872189

In [4]:
isbn = isbns[2340000]
interpark_crawl(str(isbn))

{'isbn': '9788946056909',
 '제목': '브라질 : 변화하는 사회와 새로운 과제들[양장]',
 '저자': '편박원복, 양은미',
 '역자': '없음',
 '출판일': '2014년 05월 30일',
 '출판사': '한울아카데미',
 '별점': '10',
 '리뷰': '0',
 '카테고리': '도서 > 역사와 문화 > 서양사 > 아메리카사 > 라틴아메리카사',
 '책소개': '없음',
 '저자소개': [],
 '추천사': '없음',
 '목차': '제1부 룰라-호우세피 변환기의 브라질의 미래 전망\n제 1 장 | 브라질의 부상: 진실의 양면 _에두아르두 스즈클라르즈\n제 2 장 | 브라질 사회 변화의 새로운 패턴 _마르시우 포쉬망\n제 3 장 | 브라질의 새로운 농촌 풍경: 농촌 환경의 변화와 연속 _아릴송 파바레투\n\n제2부 세계 속의 브라질\n제 4 장 | 브라질 대외정책의 기조와 변화 _이승용\n제 5 장 | 브라질 ‘파워외교’의 개념화와 실용적 의미 _김원호\n제 6 장 | 세계체제의 재설정과 브라질의 공간 _마르시우 포쉬망\n제 7 장 | 브라질-카리브 지역의 아프리칸 디아스포라와 정체성 _임소라\n\n제3부 브라질의 새로운 과제들\n제 8 장 | 브라질의 에너지 분야: 정책과 전망 _마우리시오 치옴노 토우마스캉\n제 9 장 | 신발전주의와 심해유전 개발의 과제들 _지오르지우 호마누 슈치\n제10장 | 브라질의 청정개발체제(CDM) 사업: 추진 성과와 과제 _권기수\n제11장 | 라틴아메리카 기후변화정책 발전 동인 분석: 브라질과 멕시코 비교 _하상섭\n제12장 | 아웃소싱의 법률문제: 브라질법에서의 논의를 중심으로 _조희문\n접어보기',
 '책속으로': '산업생산이 폭증하지만 인프라 투자 부족과 고평가된 환율로 고통받고 있는 곳이 브라질이다. 기록적인 수준으로 사회적 통합이 진전되고 있지만 동시에 빈민과 흑인에게는 가혹한, 수많은 폭력이 공존하는 곳이 브라질이다. 기술을 통해 투명성을 확보하고 부패를 방지하고 있지만 동시에 한 주가 

# 대량 크롤링

In [ ]:
num = 530000
results = []
count_save = num - 10000
for isbn in tqdm(isbns[num - 10000:num]):
    time.sleep(2)
    try:
        book = interpark_crawl(str(isbn))
        results.append(book)
    except:
        book = {'isbn' : str(isbn),
                '제목' : '없음',
                '저자' : '없음',
                '역자' : '없음',
                '출판일' : '없음',
                '출판사' : '없음',
                '별점' : '없음',
                '리뷰' : '없음',
                '카테고리' : '없음',
                '책소개' : '없음',
                '저자소개' : '없음',
                '추천사' : '없음',
                '목차' : '없음',
                '책속으로' : '없음',
                '출판사리뷰' : '없음',
                '판매지수' : '없음'
               }
        results.append(book)

    count_save += 1
    
    if count_save % 1000 == 0:
        file_name = f'interpark_{count_save}_2,870,000.csv'
        results_df = pd.DataFrame(results)
        results_df.to_csv('./crawled_this_week/' + file_name, encoding='utf-8-sig')
        results = []
        print(f'saving {count_save}th results to {file_name}')
        
print('Crawling Completed')

  0%|          | 0/10000 [00:00<?, ?it/s]